In [3]:
!pwd

/home/alec/code/alecmatt5/nba_betting_analysis/nba_betting_analysis/backend/notebooks


In [74]:
import pandas as pd
import numpy as np
import os
import requests
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoreadvancedv2
from datetime import datetime, timedelta
from nba_api.stats.endpoints import scoreboard

# Raw

## Get existing pickle file

In [227]:
advanced.TEAM_NAME.unique()

array(['76ers', 'Cavaliers', 'Heat', 'Grizzlies', 'Kings', 'Bulls',
       'Lakers', 'Rockets', 'Timberwolves', 'Celtics', 'Spurs',
       'Mavericks', 'Clippers', 'Warriors', 'Hornets', 'Wizards',
       'Pistons', 'Raptors', 'Nuggets', 'Pelicans', 'Thunder', 'Nets',
       'Magic', 'Suns', 'Bucks', 'Trail Blazers', 'Knicks', 'Pacers',
       'Hawks', 'Jazz'], dtype=object)

In [183]:
advanced = pd.read_pickle('../data/pkl/boxscores_advanced_team_all.pkl')
advanced

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,...,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,GAME_DATE,HOME_TEAM,PLUS_MINUS
20,22200956,1610612764,Wizards,WAS,Washington,110.1,117.2,-7.1,0.684,1.08,...,0.761,0.565,24.2,0.6,0.644,89.66,99,2023-03-04,1,-7.0
21,22200956,1610612761,Raptors,TOR,Toronto,117.2,110.1,7.1,0.638,2.14,...,0.667,0.435,14.1,0.589,0.603,89.66,99,2023-03-04,0,7.0
22,22200958,1610612748,Heat,MIA,Miami,117.0,110.1,6.9,0.738,1.55,...,0.75,0.54,20.0,0.584,0.628,99.5,100,2023-03-04,1,8.0
23,22200958,1610612737,Hawks,ATL,Atlanta,110.1,117.0,-6.9,0.595,1.38,...,0.674,0.46,16.2,0.525,0.591,99.5,99,2023-03-04,0,-8.0
24,22200959,1610612759,Spurs,SAS,San Antonio,112.2,125.8,-13.5,0.632,1.6,...,0.659,0.473,15.3,0.529,0.58,97.5,98,2023-03-04,1,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12610,21800163,1610612745,Rockets,HOU,Houston,87.9,107.7,-19.8,0.516,0.94,...,0.625,0.441,18.7,0.445,0.463,91.0,91,2018-11-08,0,-18.0
12611,21800166,1610612749,Bucks,MIL,Milwaukee,127.6,105.7,21.9,0.635,2.75,...,0.75,0.56,11.4,0.559,0.594,105.0,105,2018-11-08,0,23.0
12612,21800166,1610612744,Warriors,GSW,Golden State,105.7,127.6,-21.9,0.738,1.72,...,0.615,0.44,17.1,0.54,0.574,105.0,105,2018-11-08,1,-23.0
12613,21800165,1610612757,Trail Blazers,POR,Portland,119.6,107.1,12.4,0.45,2.57,...,0.8,0.565,7.2,0.464,0.531,97.5,97,2018-11-08,1,11.0


## Get game ids of missing games/based on dates

In [192]:
#TEMP
from datetime import datetime, timedelta
today = (datetime.utcnow() - timedelta(hours=4)).strftime('%m/%d/%Y')
dates = [today]
for i in range(1,11):
    day = (datetime.utcnow() - timedelta(days=i)).strftime('%m/%d/%Y')
    dates.append(day)
####

if len(dates) == 0:
    #from datetime import datetime, timedelta
    today = (datetime.utcnow() - timedelta(hours=4))
    yesterday = today - timedelta(days=1)
    date = yesterday.strftime('%m/%d/%Y')
    scoreboard_ = scoreboard.Scoreboard(game_date=date, league_id='00', day_offset=0)
    games_to_update = scoreboard_.game_header.get_data_frame()
else:
    games_to_update = None
    for date in dates:
        if games is None:
            scoreboard_ = scoreboard.Scoreboard(game_date=date, league_id='00', day_offset=0)
            games_to_update = scoreboard_.game_header.get_data_frame()
        else:
            scoreboard_ = scoreboard.Scoreboard(game_date=date, league_id='00', day_offset=0)
            games_to_update = pd.concat([games_to_update, scoreboard_.game_header.get_data_frame()])
if not games.empty:
    game_ids = games_to_update['GAME_ID'].unique().tolist()
else:
    game_ids = None

## Use game ids to get new advanced boxscores

In [194]:
# boxscores_advanced_player = None
boxscores_advanced_team = None
for game_id in game_ids:
    if boxscores_advanced_team is None:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced_team = gamefinder.get_data_frames()[1]
#         boxscores_advanced_player = gamefinder.get_data_frames()[0]
    else:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced_team = pd.concat([boxscores_advanced_team, gamefinder.get_data_frames()[1]])
#         boxscores_advanced_player = pd.concat([boxscores_advanced_player, gamefinder.get_data_frames()[0]])
boxscores_advanced_team
boxscores_advanced_team['GAME_ID'] = boxscores_advanced_team.GAME_ID.astype('int32')

In [195]:
boxscores_advanced_team.head(50)

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,22201035,1610612755,76ers,PHI,Philadelphia,240:00,125.7,126.9,115.1,118.5,...,21.5,0.633,0.679,1.0,0.195,94.28,92.5,77.08,93,0.579
1,22201035,1610612739,Cavaliers,CLE,Cleveland,240:00,115.1,118.5,125.7,126.9,...,13.0,0.553,0.576,1.0,0.193,94.28,92.5,77.08,92,0.421
0,22201036,1610612748,Heat,MIA,Miami,240:00,135.9,135.3,117.6,117.8,...,12.7,0.677,0.715,1.0,0.195,101.34,101.5,84.58,102,0.597
1,22201036,1610612763,Grizzlies,MEM,Memphis,240:00,117.6,117.8,135.9,135.3,...,15.8,0.604,0.612,1.0,0.198,101.34,101.5,84.58,101,0.403
0,22201037,1610612758,Kings,SAC,Sacramento,240:00,110.6,113.6,103.6,109.6,...,19.4,0.603,0.645,1.0,0.199,107.88,103.5,86.25,103,0.526
1,22201037,1610612741,Bulls,CHI,Chicago,240:00,103.6,109.6,110.6,113.6,...,14.4,0.474,0.528,1.0,0.197,107.88,103.5,86.25,104,0.474
0,22201038,1610612747,Lakers,LAL,Los Angeles,240:00,104.5,108.9,114.7,112.9,...,9.9,0.439,0.518,1.0,0.194,102.32,101.0,84.17,101,0.475
1,22201038,1610612745,Rockets,HOU,Houston,240:00,114.7,112.9,104.5,108.9,...,17.8,0.573,0.585,1.0,0.199,102.32,101.0,84.17,101,0.525
0,22201039,1610612750,Timberwolves,MIN,Minnesota,240:00,103.4,107.4,103.8,110.6,...,13.7,0.512,0.544,1.0,0.197,99.44,94.5,78.75,95,0.504
1,22201039,1610612738,Celtics,BOS,Boston,240:00,103.8,110.6,103.4,107.4,...,11.7,0.476,0.535,1.0,0.192,99.44,94.5,78.75,94,0.496


## Drop unecessary columns

In [221]:
columns = ['TEAM_NAME', 'TEAM_CITY',
    'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
    'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
    'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
    'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE',
    'PACE_PER40', 'POSS', 'PIE']

In [222]:
correct_columns = []
for column in columns:
    if column in columns_to_drop:
        continue
    else:
        correct_columns.append(column)

In [223]:
correct_columns

['TEAM_NAME',
 'TEAM_CITY',
 'OFF_RATING',
 'DEF_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TOV',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'PACE',
 'POSS']

In [196]:
columns_to_drop = ['MIN', 'E_OFF_RATING', 'E_DEF_RATING',
                   'E_NET_RATING', 'AST_RATIO', 'E_TM_TOV_PCT', 'USG_PCT',
                   'E_USG_PCT', 'E_PACE', 'PACE_PER40', 'PIE']
boxscores_advanced_team = boxscores_advanced_team.drop(columns=columns_to_drop)

## Get basic boxscores

In [197]:
nba_teams = teams.get_teams()
team_names = [team['full_name'] for team in nba_teams]
team_names.sort()
team_ids = [team['id'] for team in nba_teams]

games = None
for ids in team_ids:
    if games is None:
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=ids)
        games = gamefinder.get_data_frames()[0]
    else:
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=ids)
        games = pd.concat([games, gamefinder.get_data_frames()[0]])

games.GAME_ID = pd.to_numeric(games.GAME_ID, downcast='integer')
games.GAME_DATE = pd.to_datetime(games.GAME_DATE, infer_datetime_format=True)

if today == True:
    today = (datetime.utcnow() - timedelta(hours=4)).strftime('%Y-%m-%d')
    games = games[games['GAME_DATE'] == today]
else:
    today = (datetime.utcnow() - timedelta(hours=4)).strftime('%Y-%m-%d')
    #games = games[games['GAME_DATE'] < today]
    games = games[games['GAME_DATE'] > date].sort_values(by='GAME_DATE', ascending=False)

games.reset_index(drop=True, inplace=True)

games.sort_values(by='GAME_ID', ascending=False)
games['HOME_TEAM'] = 0
games.loc[games['MATCHUP'].str.contains('vs.'), ['HOME_TEAM']] = 1
games.loc[games['HOME_TEAM'] == 1, :]
games.sort_values(by=['GAME_DATE', 'GAME_ID'], ascending=False).reset_index(drop=True)
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM
0,22022,1610612747,LAL,Los Angeles Lakers,22201038,2023-03-15,LAL @ HOU,L,239,110,...,11.0,25.0,36.0,21,7.0,3,8,17,-4.2,0
1,22022,1610612739,CLE,Cleveland Cavaliers,22201035,2023-03-15,CLE vs. PHI,L,239,109,...,12.0,21.0,33.0,20,6.0,2,10,18,-9.0,1
2,22022,1610612746,LAC,LA Clippers,22201041,2023-03-15,LAC vs. GSW,None,227,127,...,16.0,26.0,42.0,29,10.0,1,5,16,5.8,1
3,22022,1610612741,CHI,Chicago Bulls,22201037,2023-03-15,CHI vs. SAC,L,241,114,...,13.0,35.0,48.0,28,13.0,4,14,24,-3.0,1
4,22022,1610612758,SAC,Sacramento Kings,22201037,2023-03-15,SAC @ CHI,W,241,117,...,5.0,40.0,45.0,26,6.0,3,19,19,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,22022,1610612747,LAL,Los Angeles Lakers,22200984,2023-03-07,LAL vs. MEM,W,241,112,...,7.0,50.0,57.0,27,4.0,2,13,18,9.0,1
126,22022,1610612745,HOU,Houston Rockets,22200981,2023-03-07,HOU vs. BKN,L,239,96,...,15.0,33.0,48.0,14,6.0,1,19,22,-22.0,1
127,22022,1610612744,GSW,Golden State Warriors,22200982,2023-03-07,GSW @ OKC,L,241,128,...,8.0,25.0,33.0,36,8.0,3,21,21,-9.0,0
128,22022,1610612742,DAL,Dallas Mavericks,22200983,2023-03-07,DAL vs. UTA,W,240,120,...,3.0,40.0,43.0,24,10.0,6,16,20,4.0,1


## Merge selected columns from basic to advanced

In [198]:
games_df = games[['TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM', 'PLUS_MINUS']].copy()

In [199]:
games_df

,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,1610612747,LAL,22201038,2023-03-15,0,-4.2
1,1610612739,CLE,22201035,2023-03-15,1,-9.0
2,1610612746,LAC,22201041,2023-03-15,1,5.8
3,1610612741,CHI,22201037,2023-03-15,1,-3.0
4,1610612758,SAC,22201037,2023-03-15,0,3.0
...,...,...,...,...,...,...
125,1610612747,LAL,22200984,2023-03-07,1,9.0
126,1610612745,HOU,22200981,2023-03-07,1,-22.0
127,1610612744,GSW,22200982,2023-03-07,0,-9.0
128,1610612742,DAL,22200983,2023-03-07,1,4.0


### Add dummy rows for today's games in the advanced boxscore

In [209]:
# scoreboard_ = scoreboard.Scoreboard(game_date=(datetime.utcnow() - timedelta(hours=4)).strftime('%m/%d/%Y'), league_id='00', day_offset=0)
# todays_games = scoreboard_.game_header.get_data_frame()
# todays_games_ids = todays_games.GAME_ID.tolist()

In [211]:
# todays_games

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS
0,2023-03-16T00:00:00,1,0022201042,1,7:00 pm ET,20230316/DENDET,1610612765,1610612743,2022,0,,None,Q0 -,0
1,2023-03-16T00:00:00,2,0022201043,1,7:30 pm ET,20230316/SACBKN,1610612751,1610612758,2022,0,,None,Q0 -,0
2,2023-03-16T00:00:00,3,0022201044,1,7:30 pm ET,20230316/OKCTOR,1610612761,1610612760,2022,0,,None,Q0 -,0
3,2023-03-16T00:00:00,4,0022201045,1,8:00 pm ET,20230316/INDMIL,1610612749,1610612754,2022,0,,None,Q0 -,0
4,2023-03-16T00:00:00,5,0022201046,1,10:00 pm ET,20230316/ORLPHX,1610612756,1610612753,2022,0,,NBA TV,Q0 - NBA TV,0


In [201]:
boxscores_advanced_team = boxscores_advanced_team.merge(games_df.drop(columns=['TEAM_ID']), on=['GAME_ID', 'TEAM_ABBREVIATION'])

In [202]:
boxscores_advanced_team

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,...,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,22201035,1610612755,76ers,PHI,Philadelphia,126.9,118.5,8.4,0.615,1.2,...,0.694,0.581,21.5,0.633,0.679,92.5,93,2023-03-15,0,9.0
1,22201035,1610612739,Cavaliers,CLE,Cleveland,118.5,126.9,-8.4,0.5,1.67,...,0.568,0.419,13.0,0.553,0.576,92.5,92,2023-03-15,1,-9.0
2,22201036,1610612748,Heat,MIA,Miami,135.3,117.8,17.5,0.673,2.54,...,0.735,0.524,12.7,0.677,0.715,101.5,102,2023-03-15,1,19.0
3,22201036,1610612763,Grizzlies,MEM,Memphis,117.8,135.3,-17.5,0.63,1.81,...,0.771,0.476,15.8,0.604,0.612,101.5,101,2023-03-15,0,-19.0
4,22201037,1610612758,Kings,SAC,Sacramento,113.6,109.6,4.0,0.684,1.3,...,0.677,0.472,19.4,0.603,0.645,103.5,103,2023-03-15,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,22200982,1610612744,Warriors,GSW,Golden State,118.5,126.9,-8.3,0.766,1.71,...,0.614,0.432,19.4,0.64,0.666,108.0,108,2023-03-07,0,-9.0
126,22200983,1610612762,Jazz,UTA,Utah,105.5,109.1,-3.6,0.561,1.44,...,0.863,0.519,14.5,0.527,0.563,110.0,110,2023-03-07,0,-4.0
127,22200983,1610612742,Mavericks,DAL,Dallas,109.1,105.5,3.6,0.571,1.41,...,0.8,0.481,15.5,0.579,0.608,110.0,110,2023-03-07,1,4.0
128,22200984,1610612763,Grizzlies,MEM,Memphis,96.3,104.7,-8.4,0.5,2.57,...,0.755,0.416,6.5,0.435,0.487,107.0,107,2023-03-07,0,-9.0


## Concat new boxscores to existing DF

In [219]:
#add new rows to existing data frame
advanced = pd.concat([boxscores_advanced_team, advanced]) #.reset_index(drop=True, inplace=True)
# adv_player = pd.concat([adv_player, boxscores_advanced_player])

0       2023-03-15
1       2023-03-15
2       2023-03-15
3       2023-03-15
4       2023-03-15
           ...    
12673   2018-11-08
12674   2018-11-08
12675   2018-11-08
12676   2018-11-08
12677   2018-11-08
Name: GAME_DATE, Length: 12938, dtype: datetime64[ns]

In [185]:
#drop rows that only have 1 team for the game id
value_counts = advanced['GAME_ID'].value_counts()
unique_values = value_counts[value_counts == 1].index.tolist()
advanced = advanced[~advanced['GAME_ID'].isin(unique_values)].reset_index(drop=True)

## Overwrite old pickle with new DF

In [217]:
advanced.to_pickle('../data/pkl/boxscores_advanced_team_all.pkl')
# #adv_player.to_pickle('data/pkl/boxscores_advanced_player_all.pkl')

In [220]:
advanced

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,...,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,GAME_DATE,HOME_TEAM,PLUS_MINUS
0,22201035,1610612755,76ers,PHI,Philadelphia,126.9,118.5,8.4,0.615,1.2,...,0.694,0.581,21.5,0.633,0.679,92.5,93,2023-03-15,0,9.0
1,22201035,1610612739,Cavaliers,CLE,Cleveland,118.5,126.9,-8.4,0.5,1.67,...,0.568,0.419,13.0,0.553,0.576,92.5,92,2023-03-15,1,-9.0
2,22201036,1610612748,Heat,MIA,Miami,135.3,117.8,17.5,0.673,2.54,...,0.735,0.524,12.7,0.677,0.715,101.5,102,2023-03-15,1,19.0
3,22201036,1610612763,Grizzlies,MEM,Memphis,117.8,135.3,-17.5,0.63,1.81,...,0.771,0.476,15.8,0.604,0.612,101.5,101,2023-03-15,0,-19.0
4,22201037,1610612758,Kings,SAC,Sacramento,113.6,109.6,4.0,0.684,1.3,...,0.677,0.472,19.4,0.603,0.645,103.5,103,2023-03-15,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12673,21800163,1610612745,Rockets,HOU,Houston,87.9,107.7,-19.8,0.516,0.94,...,0.625,0.441,18.7,0.445,0.463,91.0,91,2018-11-08,0,-18.0
12674,21800166,1610612749,Bucks,MIL,Milwaukee,127.6,105.7,21.9,0.635,2.75,...,0.75,0.56,11.4,0.559,0.594,105.0,105,2018-11-08,0,23.0
12675,21800166,1610612744,Warriors,GSW,Golden State,105.7,127.6,-21.9,0.738,1.72,...,0.615,0.44,17.1,0.54,0.574,105.0,105,2018-11-08,1,-23.0
12676,21800165,1610612757,Trail Blazers,POR,Portland,119.6,107.1,12.4,0.45,2.57,...,0.8,0.565,7.2,0.464,0.531,97.5,97,2018-11-08,1,11.0


# Preprocess

## Get advanced boxscores from pickle

In [ ]:
#advanced = pd.read_pickle('../data/pkl/boxscores_advanced_team_all.pkl')

## Preprocess the data

### Roll the columns

In [213]:
### ROLLING FUNCTION
def roll(df, roll_number = 10, procedure = '', suff = '_Roll', selected_columns=[]):
    df_rolling = df[selected_columns + ["TEAM_ABBREVIATION"]]
    df_rolling = df_rolling.groupby(["TEAM_ABBREVIATION"], group_keys=False)

    def find_team_averages(team):
        return team.rolling(roll_number, closed='left').mean()

    def find_team_medians(team):
        return team.rolling(roll_number, closed='left').median()

    def find_team_stds(team):
        return team.rolling(roll_number, closed='left').std()

    if procedure == 'median':
        df_rolling = df_rolling.apply(find_team_medians)
    elif procedure == 'std':
        df_rolling = df_rolling.apply(find_team_stds)
    else:
        procedure = 'mean'
        df_rolling = df_rolling.apply(find_team_averages)

    df_rolling = df_rolling[selected_columns]
    df_rolling = df_rolling.sort_index()

    new_column_names = {}
    for col in df_rolling.columns:
        new_column_names[col] = col + suff + '_' + procedure

    df_rolling = df_rolling.rename(columns=new_column_names)
    return df_rolling

#### SORT THE COLUMNS BEFORE PASSING TO ROLL FUNCTION

In [ ]:
advanced_desc = advanced.sort_values(by=['GAME_DATE'], ascending=True).copy()

In [ ]:
non_eng_features = ['TEAM_ABBREVIATION', 'GAME_ID', 'TEAM_ID', 'TEAM_NAME',
                    'GAME_DATE', 'HOME_TEAM', 'PLUS_MINUS']
eng_features = advanced_desc.drop(columns=non_eng_features).columns.tolist()

In [ ]:
#caluculate rolling metrics
if 'mean' in roll_methods:
    df_temp = roll(df = advanced_desc, roll_number=roll_number, procedure='mean', selected_columns=eng_features)
    advanced = advanced.merge(df_temp, left_index=True, right_index=True)
if 'median' in roll_methods:
    df_temp = roll(df = advanced_desc, roll_number=roll_number, procedure='median', selected_columns=eng_features)
    advanced = advanced.merge(df_temp, left_index=True, right_index=True)
if 'std' in roll_methods:
    df_temp = roll(df = advanced_desc, roll_number=roll_number, procedure='std', selected_columns=eng_features)
    advanced = advanced.merge(df_temp, left_index=True, right_index=True)

### Drop orginal columns

In [ ]:
#drop original columns to prevent data leakage
drop_columns = ['OFF_RATING', 'DEF_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'OREB_PCT', 'DREB_PCT',
    'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'PACE', 'POSS']
advanced.drop(columns=drop_columns, inplace=True)

### Split between home and away, rename columns, merge back together

In [ ]:
#split data frame between the home teams and the away teams
advanced = advanced.sort_values(by=['GAME_DATE', 'GAME_ID', 'HOME_TEAM'], ascending=False).reset_index(drop=True)
adv_home = advanced.iloc[::2].copy()
adv_away = advanced.iloc[1::2].copy()

#change the column names based on home or away
columns_away = {}
columns_home = {}
columns_to_merge = []
for column in advanced.columns:
    if column == 'GAME_ID' or column == 'PLUS_MINUS' or column == 'GAME_DATE':
        continue
    columns_to_merge.append(column + '_a')
    columns_away[column] = column + '_a'
    columns_home[column] = column + '_h'
#merge the home and away data frames on to the same game id
adv_away.rename(columns=columns_away, inplace=True)
adv_home.rename(columns=columns_home, inplace=True)
columns_to_merge.append('GAME_ID')
merged_df = adv_home.merge(adv_away[columns_to_merge], on=['GAME_ID'])
merged_df = merged_df.dropna()

### Add elo ratings

In [ ]:
#ELO FUNCTION
def update_elo():
    today = (datetime.utcnow() - timedelta(hours=4)).strftime('%Y-%m-%d')
    URL = 'https://projects.fivethirtyeight.com/nba-model/nba_elo.csv'
    elo_past = pd.read_csv(URL)
    elo_past['date'] = pd.to_datetime(elo_past['date'])
    elo_past= elo_past[elo_past['date'] > '2018-09-01']
    elo_past[elo_past['date'] < today]
    map = {'BRK': 'BKN', 'CHO': 'CHA', 'PHO': 'PHX'}
    elo_past = elo_past.replace({'team1': map, 'team2': map})
    elo_past = elo_past[['date', 'team1', 'team2', 'elo1_pre', 'elo2_pre', 'raptor1_pre', 'raptor2_pre']]
    elo_past.to_pickle('data/pkl/elo_past.pkl')
    return

In [ ]:
#get elo and raptor scores
update_elo()
elo_past = pd.read_pickle('data/pkl/elo_past.pkl')


merged_df = merged_df.merge(elo_past, left_on=['GAME_DATE', 'TEAM_ABBREVIATION_h'], right_on=['date', 'team1'])
merged_df.drop(columns=['date', 'team1', 'team2'], inplace=True)
merged_df.rename(columns={'elo1_pre': 'elo_h', 'elo2_pre': 'elo_a', 'raptor1_pre': 'raptor_h', 'raptor2_pre': 'raptor_a'},
      inplace=True)

### Scale

In [ ]:
###TEMP
scaled = False
#####

#make lists of feature column names
X_features_num = [col for col in merged_df.columns if 'GAME_ID' not in col
                 and 'GAME_DATE' not in col
                 and 'TEAM_CITY' not in col
                 and 'TEAM_ID' not in col
                 and 'TEAM_NAME' not in col
                 and 'TEAM_ABBREVIATION' not in col
                 and 'PLUS_MINUS' not in col
                 and 'HOME_TEAM' not in col]

X_features_cat = ['TEAM_ABBREVIATION_h', 'TEAM_ABBREVIATION_a']
#scale the numerical features
preproc_data = merged_df.copy()
if scaled == True:
    scaler = MinMaxScaler()
    preproc_data[X_features_num] = scaler.fit_transform(preproc_data[X_features_num])

### OHE

In [ ]:
###TEMP
ohe = True
#####

#one hot encode the teams
if ohe == True:
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(preproc_data[X_features_cat])
    cols = [str(team) +'_h' for team in ohe.categories_[0]] + [str(team) +'_a' for team in ohe.categories_[1]]
    preproc_data[cols]=ohe.transform(preproc_data[X_features_cat])
    X_features = [col for col in preproc_data.columns if 'GAME_ID' not in col
                 and 'GAME_DATE' not in col
                 and 'TEAM_CITY' not in col
                 and 'TEAM_ID' not in col
                 and 'TEAM_NAME' not in col
                 and 'TEAM_ABBREVIATION' not in col
                 and 'PLUS_MINUS' not in col
                 and 'HOME_TEAM' not in col]
else:
    X_features = [col for col in preproc_data.columns if 'GAME_ID' not in col
                 and 'GAME_DATE' not in col
                 and 'TEAM_ID' not in col
                 and 'TEAM_NAME' not in col
                 and 'TEAM_ABBREVIATION' not in col
                 and 'PLUS_MINUS' not in col
                 and 'HOME_TEAM' not in col]

## Make a copy of the existing preprocess pickle file

In [ ]:
yesterday_

## Overwrite the existing preprocess pickle file

['03/15/2023',
 '03/14/2023',
 '03/13/2023',
 '03/12/2023',
 '03/11/2023',
 '03/10/2023',
 '03/09/2023',
 '03/08/2023',
 '03/07/2023',
 '03/06/2023']

In [76]:
advanced = pd.read_pickle('../data/pkl/boxscores_advanced_team_all.pkl')

In [80]:
nba_teams = teams.get_teams()
team_names = [team['full_name'] for team in nba_teams]
team_names.sort()
team_ids = [team['id'] for team in nba_teams]

games = None
for ids in team_ids:
    if games is None:
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=ids)
        games = gamefinder.get_data_frames()[0]
    else:
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=ids)
        games = pd.concat([games, gamefinder.get_data_frames()[0]])

games.GAME_ID = pd.to_numeric(games.GAME_ID, downcast='integer')
games.GAME_DATE = pd.to_datetime(games.GAME_DATE, infer_datetime_format=True)

#Temporary
today = False
date = "2018-09-01"
#############

if today == True:
    today_date = (datetime.utcnow() - timedelta(hours=4)).strftime('%Y-%m-%d')
    games = games[games['GAME_DATE'] == today_date]
else:
    today_date = (datetime.utcnow() - timedelta(hours=4)).strftime('%Y-%m-%d')
    games = games[games['GAME_DATE'] < today_date]
    games = games[games['GAME_DATE'] > date].sort_values(by='GAME_DATE', ascending=False)

games.reset_index(drop=True, inplace=True)

games.sort_values(by='GAME_ID', ascending=False)
games['HOME_TEAM'] = 0
games.loc[games['MATCHUP'].str.contains('vs.'), ['HOME_TEAM']] = 1
games.loc[games['HOME_TEAM'] == 1, :]
games.sort_values(by=['GAME_DATE', 'GAME_ID'], ascending=False).reset_index(drop=True)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME_TEAM
0,22022,1610612752,NYK,New York Knicks,22201034,2023-03-14,NYK @ POR,W,240,123,...,9.0,43.0,52.0,23,8.0,8,12,13,16.0,0
1,22022,1610612757,POR,Portland Trail Blazers,22201034,2023-03-14,POR vs. NYK,L,239,107,...,11.0,27.0,38.0,22,8.0,3,10,19,-16.0,1
2,22022,1610612749,MIL,Milwaukee Bucks,22201033,2023-03-14,MIL @ PHX,W,240,116,...,13.0,33.0,46.0,24,8.0,6,12,15,12.0,0
3,22022,1610612756,PHX,Phoenix Suns,22201033,2023-03-14,PHX vs. MIL,L,239,104,...,8.0,33.0,41.0,23,7.0,3,11,26,-12.0,1
4,22022,1610612753,ORL,Orlando Magic,22201032,2023-03-14,ORL @ SAS,L,240,114,...,13.0,29.0,42.0,24,7.0,2,9,16,-18.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13207,12018,1610612761,TOR,Toronto Raptors,11800003,2018-09-29,TOR vs. POR,W,238,122,...,12.0,33.0,45.0,15,8.0,6,15,28,18.0,1
13208,12018,1610612757,POR,Portland Trail Blazers,11800003,2018-09-29,POR @ TOR,L,239,104,...,11.0,35.0,46.0,25,4.0,2,24,45,-18.0,0
13209,12018,1610612738,BOS,Boston Celtics,11800002,2018-09-28,BOS @ CHA,L,238,97,...,11.0,45.0,56.0,12,9.0,7,12,35,-7.0,0
13210,12018,1610612766,CHA,Charlotte Hornets,11800002,2018-09-28,CHA vs. BOS,W,241,104,...,12.0,53.0,65.0,17,4.0,7,17,22,7.0,1


In [81]:
if len(dates) == 0:
    #from datetime import datetime, timedelta
    today = (datetime.utcnow() - timedelta(hours=4))
    yesterday = today - timedelta(days=1)
    date = yesterday.strftime('%m/%d/%Y')
    scoreboard_ = scoreboard.Scoreboard(game_date=date, league_id='00', day_offset=0)
    games_to_update = scoreboard_.game_header.get_data_frame()
else:
    games_to_update = None
    for date in dates:
        if games is None:
            scoreboard_ = scoreboard.Scoreboard(game_date=dates, league_id='00', day_offset=0)
            games_to_update = scoreboard_.game_header.get_data_frame()
        else:
            scoreboard_ = scoreboard.Scoreboard(game_date=dates, league_id='00', day_offset=0)
            games_to_update = pd.concat([games_to_update, scoreboard_.game_header.get_data_frame()])
if not games.empty:
    game_ids = list(games_to_update['GAME_ID'])
else:
    game_ids = None

In [83]:
adv_team

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,...,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,GAME_DATE,HOME_TEAM,PLUS_MINUS
20,22200956,1610612764,Wizards,WAS,Washington,110.1,117.2,-7.1,0.684,1.08,...,0.761,0.565,24.2,0.6,0.644,89.66,99,2023-03-04,1,-7.0
21,22200956,1610612761,Raptors,TOR,Toronto,117.2,110.1,7.1,0.638,2.14,...,0.667,0.435,14.1,0.589,0.603,89.66,99,2023-03-04,0,7.0
22,22200958,1610612748,Heat,MIA,Miami,117.0,110.1,6.9,0.738,1.55,...,0.75,0.54,20.0,0.584,0.628,99.5,100,2023-03-04,1,8.0
23,22200958,1610612737,Hawks,ATL,Atlanta,110.1,117.0,-6.9,0.595,1.38,...,0.674,0.46,16.2,0.525,0.591,99.5,99,2023-03-04,0,-8.0
24,22200959,1610612759,Spurs,SAS,San Antonio,112.2,125.8,-13.5,0.632,1.6,...,0.659,0.473,15.3,0.529,0.58,97.5,98,2023-03-04,1,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12610,21800163,1610612745,Rockets,HOU,Houston,87.9,107.7,-19.8,0.516,0.94,...,0.625,0.441,18.7,0.445,0.463,91.0,91,2018-11-08,0,-18.0
12611,21800166,1610612749,Bucks,MIL,Milwaukee,127.6,105.7,21.9,0.635,2.75,...,0.75,0.56,11.4,0.559,0.594,105.0,105,2018-11-08,0,23.0
12612,21800166,1610612744,Warriors,GSW,Golden State,105.7,127.6,-21.9,0.738,1.72,...,0.615,0.44,17.1,0.54,0.574,105.0,105,2018-11-08,1,-23.0
12613,21800165,1610612757,Trail Blazers,POR,Portland,119.6,107.1,12.4,0.45,2.57,...,0.8,0.565,7.2,0.464,0.531,97.5,97,2018-11-08,1,11.0


In [84]:
games_df = games[['TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM', 'PLUS_MINUS']].copy()
columns_to_drop = ['MIN', 'E_OFF_RATING', 'E_DEF_RATING',
                   'E_NET_RATING', 'AST_RATIO', 'E_TM_TOV_PCT', 'USG_PCT',
                   'E_USG_PCT', 'E_PACE', 'PACE_PER40', 'PIE']
adv_team = adv_team.drop(columns=columns_to_drop)

#change game_id type to match between the 2 data frames
adv_team['GAME_ID'] = adv_team['GAME_ID'].astype('int32')

#merge the needed columns from basic to advanced
adv_team = adv_team.merge(games_df.drop(columns=['TEAM_ID']), on=['GAME_ID', 'TEAM_ABBREVIATION'])

In [85]:
# boxscores_advanced_player = None
boxscores_advanced_team = None
for game_id in game_ids:
    if boxscores_advanced_team is None:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced_team = gamefinder.get_data_frames()[1]
#         boxscores_advanced_player = gamefinder.get_data_frames()[0]
    else:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced_team = pd.concat([boxscores_advanced_team, gamefinder.get_data_frames()[1]])
#         boxscores_advanced_player = pd.concat([boxscores_advanced_player, gamefinder.get_data_frames()[0]])

#add new rows to existing data frame
adv_team = pd.concat([boxscores_advanced_team, adv_team]).reset_index(drop=True, inplace=True)
# adv_player = pd.concat([adv_player, boxscores_advanced_player])
adv_team

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,PACE,PACE_PER40,POSS,PIE,GAME_DATE_x,HOME_TEAM_x,PLUS_MINUS_x,GAME_DATE_y,HOME_TEAM_y,PLUS_MINUS_y
0,0022201035,1610612755,76ers,PHI,Philadelphia,240:00,125.7,126.9,115.1,118.5,...,92.5,77.08,93,0.579,NaT,NaN,NaN,NaT,NaN,NaN
1,0022201035,1610612739,Cavaliers,CLE,Cleveland,240:00,115.1,118.5,125.7,126.9,...,92.5,77.08,92,0.421,NaT,NaN,NaN,NaT,NaN,NaN
0,0022201036,1610612748,Heat,MIA,Miami,240:00,135.9,135.3,117.6,117.8,...,101.5,84.58,102,0.597,NaT,NaN,NaN,NaT,NaN,NaN
1,0022201036,1610612763,Grizzlies,MEM,Memphis,240:00,117.6,117.8,135.9,135.3,...,101.5,84.58,101,0.403,NaT,NaN,NaN,NaT,NaN,NaN
0,0022201037,1610612758,Kings,SAC,Sacramento,240:00,110.6,113.6,103.6,109.6,...,103.5,86.25,103,0.526,NaT,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12610,21800163,1610612745,Rockets,HOU,Houston,NaN,NaN,87.9,NaN,107.7,...,91.0,NaN,91,NaN,2018-11-08,0.0,-18.0,2018-11-08,0.0,-18.0
12611,21800166,1610612749,Bucks,MIL,Milwaukee,NaN,NaN,127.6,NaN,105.7,...,105.0,NaN,105,NaN,2018-11-08,0.0,23.0,2018-11-08,0.0,23.0
12612,21800166,1610612744,Warriors,GSW,Golden State,NaN,NaN,105.7,NaN,127.6,...,105.0,NaN,105,NaN,2018-11-08,1.0,-23.0,2018-11-08,1.0,-23.0
12613,21800165,1610612757,Trail Blazers,POR,Portland,NaN,NaN,119.6,NaN,107.1,...,97.5,NaN,97,NaN,2018-11-08,1.0,11.0,2018-11-08,1.0,11.0


In [88]:
boxscores_advanced_team

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022201035,1610612755,76ers,PHI,Philadelphia,240:00,125.7,126.9,115.1,118.5,...,21.5,0.633,0.679,1.0,0.195,94.28,92.5,77.08,93,0.579
1,0022201035,1610612739,Cavaliers,CLE,Cleveland,240:00,115.1,118.5,125.7,126.9,...,13.0,0.553,0.576,1.0,0.193,94.28,92.5,77.08,92,0.421
0,0022201036,1610612748,Heat,MIA,Miami,240:00,135.9,135.3,117.6,117.8,...,12.7,0.677,0.715,1.0,0.195,101.34,101.5,84.58,102,0.597
1,0022201036,1610612763,Grizzlies,MEM,Memphis,240:00,117.6,117.8,135.9,135.3,...,15.8,0.604,0.612,1.0,0.198,101.34,101.5,84.58,101,0.403
0,0022201037,1610612758,Kings,SAC,Sacramento,240:00,110.6,113.6,103.6,109.6,...,19.4,0.603,0.645,1.0,0.199,107.88,103.5,86.25,103,0.526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0022201039,1610612738,Celtics,BOS,Boston,None,None,0.0,None,0.0,...,None,None,None,1.0,0.000,None,None,None,0,None
0,0022201040,1610612759,Spurs,SAS,San Antonio,None,None,0.0,None,0.0,...,None,None,None,1.0,0.000,None,None,None,0,None
1,0022201040,1610612742,Mavericks,DAL,Dallas,None,None,0.0,None,0.0,...,None,None,None,1.0,0.000,None,None,None,0,None
0,0022201041,1610612746,Clippers,LAC,LA,None,None,0.0,None,0.0,...,None,None,None,1.0,0.000,None,None,None,0,None


In [ ]:
# #update the pickle file with all the data
# adv_team.to_pickle('data/pkl/boxscores_advanced_team_all.pkl')
# # adv_player.to_pickle('data/pkl/boxscores_advanced_player_all.pkl')

In [43]:
# fixed = advanced[advanced['GAME_DATE'] < '2023-03-05']
# fixed.to_pickle('../data/pkl/boxscores_advanced_team_all.pkl')

In [14]:
#drop rows that only have 1 team for the game id
value_counts = advanced['GAME_ID'].value_counts()
unique_values = value_counts[value_counts == 1].index.tolist()
advanced = advanced[~advanced['GAME_ID'].isin(unique_values)].reset_index(drop=True)
advanced_desc = advanced.sort_values(by=['GAME_DATE'], ascending=True).copy()

In [19]:
advanced.sort_values('GAME_DATE').head(50)

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,OFF_RATING,DEF_RATING,NET_RATING,AST_PCT,AST_TOV,...,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,POSS,GAME_DATE,HOME_TEAM,PLUS_MINUS
12581,21800165,1610612746,Clippers,LAC,LA,107.1,119.6,-12.4,0.463,2.71,...,0.638,0.435,7.1,0.506,0.537,97.5,98,2018-11-08,0,-11.0
12580,21800165,1610612757,Trail Blazers,POR,Portland,119.6,107.1,12.4,0.45,2.57,...,0.8,0.565,7.2,0.464,0.531,97.5,97,2018-11-08,1,11.0
12579,21800166,1610612744,Warriors,GSW,Golden State,105.7,127.6,-21.9,0.738,1.72,...,0.615,0.44,17.1,0.54,0.574,105.0,105,2018-11-08,1,-23.0
12578,21800166,1610612749,Bucks,MIL,Milwaukee,127.6,105.7,21.9,0.635,2.75,...,0.75,0.56,11.4,0.559,0.594,105.0,105,2018-11-08,0,23.0
12577,21800163,1610612745,Rockets,HOU,Houston,87.9,107.7,-19.8,0.516,0.94,...,0.625,0.441,18.7,0.445,0.463,91.0,91,2018-11-08,0,-18.0
12576,21800163,1610612760,Thunder,OKC,Oklahoma City,107.7,87.9,19.8,0.5,1.2,...,0.722,0.559,16.5,0.5,0.535,91.0,91,2018-11-08,1,18.0
12575,21800164,1610612738,Celtics,BOS,Boston,105.5,100.0,5.5,0.538,1.31,...,0.852,0.574,14.5,0.474,0.524,99.17,110,2018-11-08,0,7.0
12574,21800164,1610612756,Suns,PHX,Phoenix,100.0,105.5,-5.5,0.61,1.67,...,0.672,0.426,13.8,0.5,0.538,99.17,109,2018-11-08,1,-7.0
12561,21800167,1610612753,Magic,ORL,Orlando,117.0,106.9,10.1,0.591,1.63,...,0.767,0.529,16.0,0.587,0.62,100.5,100,2018-11-09,1,9.0
12562,21800169,1610612765,Pistons,DET,Detroit,118.1,103.8,14.3,0.652,2.73,...,0.766,0.466,10.5,0.577,0.588,105.0,105,2018-11-09,0,15.0
